## Building A Chatbot
We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. 

In [4]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [6]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "llama-3.3-70b-versatile")

In [8]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hello my name is Safeen Khan and I work at Accenture")])

AIMessage(content="Hello Safeen Khan! It's nice to meet you. Accenture is a well-known and respected company, so that's great that you're a part of their team. What do you do at Accenture, if you don't mind me asking? Are you working on any exciting projects or initiatives?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 48, 'total_tokens': 110, 'completion_time': 0.182112131, 'prompt_time': 0.002139561, 'queue_time': 0.055247258, 'total_time': 0.184251692}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d1387737-47cd-4539-a554-3f09bf4e2ac7-0', usage_metadata={'input_tokens': 48, 'output_tokens': 62, 'total_tokens': 110})

In [10]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hello my name is Safeen Khan and I work at Accenture"),
        AIMessage(content = "Hello Safeen Khan! It's nice to meet you. Accenture is a well-known and respected company, so that's great that you're a part of their team. What do you do at Accenture, if you don't mind me asking? Are you working on any exciting projects or initiatives?"),
        HumanMessage(content = "What is my name and where do I work")
    ]
)

AIMessage(content='Your name is Safeen Khan, and you work at Accenture.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 128, 'total_tokens': 143, 'completion_time': 0.031910061, 'prompt_time': 0.006805959, 'queue_time': 0.0536388, 'total_time': 0.03871602}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9b795353-f551-46b9-bf51-d0bb49e43703-0', usage_metadata={'input_tokens': 128, 'output_tokens': 15, 'total_tokens': 143})

We can see the model has some history and can remember the content that we have told

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [15]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id : str) ->BaseChatMessageHistory:    #BaseChatMessageHistory is the return type
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [16]:
config = {"configurable":{
    "session_id" : "chat1"
}}

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello my name is Safeen Khan and I work at Accenture")],
    config = config)

In [18]:
response.content

"Hello Safeen Khan! It's nice to meet you. Welcome to our conversation. Accenture is a well-known and respected company, what do you do there? Are you working on any exciting projects or initiatives? I'd love to hear more about your role and experiences."

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my name ?")],
    config = config)
response.content

'Your name is Safeen Khan, and you work at Accenture.'

Now if we change the config or main the session_id we will se it doen't remembers my name

In [26]:
config2 = {"configurable":{
    "session_id":"chat2"
}}

In [27]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my name ?")],
    config = config2
)
response.content

"I don't know your name. I'm a large language model, I don't have any information about you, including your name. Our conversation just started, and I don't have any prior knowledge about you. If you'd like to share your name, I'd be happy to chat with you!"